# 크롤링해야 되는 것
#### -> 자산총계, 부채총계, 자본총계 = 재무상태표
#### -> 매출액, 영업이익, 당기순이익(분기순이익) = 포괄손익계산서
api_key = 'ff12d16bfd2cb5ddef36494b71c419fda68b304d'


- 해당 분기보고서가 없을 때 예외처리를 생각해보기

### 기업들의 고유번호 받아오기

In [172]:
!pip install dart_fss

### 1. 패키지 설치 및 import + 비상장, 상장 기업 정보 가져오기

In [173]:
import dart_fss as dart
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import zipfile
import io
from bs4 import BeautifulSoup
api_key = 'ff12d16bfd2cb5ddef36494b71c419fda68b304d'
dart.set_api_key(api_key=api_key)

# 상장+비상장 기업명 크롤링
corp_list = dart.api.filings.get_corp_code()
corp_df = pd.DataFrame.from_dict(corp_list)
# corp_df = corp_df.dropna(subset = 'stock_code').sort_values('modify_date',ascending=False).reset_index(drop=True)
corp_df = pd.DataFrame.from_dict(corp_list).sort_values('modify_date', ascending=False).reset_index(drop=True)
corp_df['done_YN'] = "N"
corp_df.columns = ['고유번호', '회사명', '종목코드', '수정날짜', 'done_YN']
corp_df.drop(columns = ['done_YN'], inplace = True)
display(corp_df)

,고유번호,회사명,종목코드,수정날짜
0,00812308,미림아트텍,None,20250117
1,01340871,에코비트워터,None,20250117
2,00138792,아시아나항공,020560,20250117
3,00141389,영풍정밀,036560,20250117
4,00151809,충청에너지서비스,None,20250117
...,...,...,...,...
109395,00417547,스피어헤드,None,20170630
109396,00274298,신한산전,None,20170630
109397,00330886,쏘텍코리아,None,20170630
109398,00309017,씨아이브레인,None,20170630


### 2. 특정 기업 필터링

In [174]:
# 주어진 company_dic
# company_dic = {"SGA": "049470", "SGA솔루션즈":"184230", "안랩":"053800", "시큐브":"131090", "윈스":"136540", "이글루":"067920",
#                "한컴위드":"054920", "네오리진": "094860", "케이사인":"192250", "이스트소프트":"047560", "라온시큐어": "042510","파수":"150900",
#                "한국정보인증": "053300", "지란지교시큐리티":"208350","수산아이앤티":"050960", "지니언스": "263860", "드림시큐리티": "203650",
#                "모니터랩":"434480", "파이오링크":"170790", "시큐센":"232830", "시큐레터": "418250",
#                "다우기술":"023590", "다우데이타":"032190", "한글과컴퓨터": "030520", "MDS테크":"086960", "아이티센":"124500", "콤텍시스템": "031820", "소프트센":"032680",
#                "대신정보통신":"020180", "쌍용정보통신":"010280", "에스넷": "038680"
#                }

company_dic = {"SGA": "049470", "다우데이타":"032190", "한글과컴퓨터": "030520", "MDS테크":"086960", "아이티센":"124500", "콤텍시스템": "031820",
                "쌍용정보통신":"010280", "토마토시스템": "393210", "오파스넷": "173130", "오픈베이스": "049480"}

# company_dic의 종목코드 리스트
target_stock_codes = list(company_dic.values())

# DataFrame에서 종목코드 필터링
filtered_df = corp_df[corp_df['종목코드'].isin(target_stock_codes)]


# company_dic의 keys에 따라 순서를 맞춘 DataFrame 생성
sorted_filtered_df = pd.DataFrame(columns=filtered_df.columns)

for key in company_dic.keys():
    stock_code = company_dic[key]
    matching_row = filtered_df[filtered_df['종목코드'] == stock_code]
    if not matching_row.empty:
        sorted_filtered_df = pd.concat([sorted_filtered_df, matching_row])

# 결과 출력
# first_df = sorted_filtered_df.iloc[:2]
# voice_eye = corp_df[corp_df['회사명']=='보이스아이']
# rest_df = sorted_filtered_df.iloc[2:]
# sorted_filtered_df = pd.concat([first_df, voice_eye,rest_df], ignore_index=True)
display(sorted_filtered_df)

,고유번호,회사명,종목코드,수정날짜
3656,00351579,SGA,049470,20240801
11094,00238782,다우데이타,032190,20240401
34795,00204262,한글과컴퓨터,030520,20230214
21423,00445841,MDS테크,086960,20230410
11190,00620868,아이티센,124500,20240401
11043,00186559,콤텍시스템,031820,20240402
541,00138303,쌍용정보통신,010280,20241220
7225,01436479,토마토시스템,393210,20240619
29056,00867849,오파스넷,173130,20230221
37515,00335076,오픈베이스,049480,20230105


### 3. API 요청을 보내고 JSON 데이터를 받아 DataFrame으로 변환하는 함수

In [175]:
def Frame(url, items, item_names, params):
    """
    url : json형태로 요청하는 주소
    items : 반환되는 데이터들의 key를 가진 리스트
    item_names :  데이터프레임을 만들때 컬럼명 리스트
    params : url 요청시 필수값으로 들어가는 인자들을 가진 딕셔너리
    """
    res = requests.get(url, params)
    json_data = res.json()
    json_dict = json.loads(res.text)
    data = []
    if json_dict['status'] == "000":  # 오류 없이 정상적으로 데이터가 있다면
        for line in json_dict['list']:
            data.append([])
            for itm in items:
                if itm in line.keys():
                    data[-1].append(line[itm])
                else:
                    data[-1].append('')
    df = pd.DataFrame(data, columns=item_names)
    return df

### 4. 단일 회사 재무제표 조회 함수
- crtfc_key = 개인 api 키
- corp_code = 기업 고유번호
- bsns_year = 해당 연도
- reprt_code =  보고서 종류(표 참고)
- fs_div = 재무제표 구분(연결 = "CFS" or 개별 = "OFS")

| 보고서 종류  | reprt_code 값 |
|-------------|--------------|
| 사업보고서  | `11011` |
| 반기보고서  | `11012` |
| 1분기보고서 | `11013` |
| 3분기보고서 | `11014` |

In [176]:
def get_단일회사재무제표(crtfc_key, corp_code, bsns_year, reprt_code, fs_div = "CFS"):
    items = ["rcept_no","reprt_code","bsns_year","corp_code","sj_div","sj_nm",
             "account_id","account_nm","account_detail","thstrm_nm", "thstrm_amount",
             "thstrm_add_amount","frmtrm_nm","frmtrm_amount", "frmtrm_q_nm","frmtrm_q_amount",
             "frmtrm_add_amount","bfefrmtrm_nm", "bfefrmtrm_amount","ord"]
    item_names = ["접수번호","보고서코드","사업연도","고유번호","재무제표구분", "재무제표명",
                  "계정ID","계정명","계정상세","당기명","당기금액", "당기누적금액","전기명","전기금액","전기명(분/반기)",
                  "전기금액(분/반기)","전기누적금액","전전기명","전전기금액", "계정과목정렬순서"]
    params = {'crtfc_key':crtfc_key, 'corp_code':corp_code, 'bsns_year':bsns_year, 'reprt_code':reprt_code, 'fs_div':fs_div}
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?"
    df = Frame(url, items, item_names, params)
    return df

### 5. 전체 기업 재무제표 수집
- 2024 반기 정상 작동
- 2024 1분기 정상 작동
- 2024 3분기 대신정보통신 빠짐 -> 없어도 되게 함

In [177]:
corp_code = list(sorted_filtered_df['고유번호'])
corp_code[0]

'00351579'

In [178]:
import json


# corp_code = sorted_filtered_df['고유번호']
final_reprt_df = get_단일회사재무제표(api_key, corp_code[0], 2023, '11012', fs_div = "OFS")
#반복
from tqdm import tqdm
for k in tqdm(range(1, len(sorted_filtered_df))):
    reprt_df = get_단일회사재무제표(api_key, corp_code[k], 2023, '11012', fs_div = "OFS")
    final_reprt_df = pd.concat([final_reprt_df, reprt_df], axis = 0)

100%|██████████| 9/9 [00:10<00:00,  1.17s/it]


In [179]:
final_reprt_df

,접수번호,보고서코드,사업연도,고유번호,재무제표구분,재무제표명,계정ID,계정명,계정상세,당기명,당기금액,당기누적금액,전기명,전기금액,전기명(분/반기),전기금액(분/반기),전기누적금액,전전기명,전전기금액,계정과목정렬순서
0,20230811002512,11012,2023,00351579,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 27 기 반기말,21056567798,,제 26 기말,25515476742,,,,,,1
1,20230811002512,11012,2023,00351579,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 27 기 반기말,112003998,,제 26 기말,4725764715,,,,,,2
2,20230811002512,11012,2023,00351579,BS,재무상태표,ifrs-full_TradeAndOtherCurrentReceivables,매출채권 및 기타유동채권,-,제 27 기 반기말,14942394354,,제 26 기말,10032132051,,,,,,3
3,20230811002512,11012,2023,00351579,BS,재무상태표,ifrs-full_CurrentFinancialAssetsAtFairValueThr...,당기손익-공정가치금융자산,-,제 27 기 반기말,1056584869,,제 26 기말,1199590459,,,,,,4
4,20230811002512,11012,2023,00351579,BS,재무상태표,ifrs-full_OtherCurrentFinancialAssets,기타금융자산,-,제 27 기 반기말,1340514405,,제 26 기말,6385742957,,,,,,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,20230814001719,11012,2023,00335076,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|자본금 [member],제 32 기 반기,15711191500,,,,제 31 기 반기,15711191500,,,,7
102,20230814001719,11012,2023,00335076,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|이익잉여금 [member],제 32 기 반기,26309405584,,,,제 31 기 반기,23464074299,,,,7
103,20230814001719,11012,2023,00335076,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|기타자본구성요소 [member],제 32 기 반기,635677911,,,,제 31 기 반기,619201837,,,,7
104,20230814001719,11012,2023,00335076,SCE,자본변동표,ifrs-full_Equity,기말자본,별도재무제표 [member],제 32 기 반기,66880998419,,,,제 31 기 반기,63963792764,,,,7


### 6. 재무제표별 데이터 분리

In [180]:
def sep(x, df):
    df = df[df['재무제표구분'] == x]
    return df

final_BS = sep('BS', final_reprt_df) #재무상태표
final_CIS = sep('CIS', final_reprt_df) #포괄손익계산서
final_CF = sep('CF', final_reprt_df) #현금흐름표
final_SCE = sep('SCE', final_reprt_df) #자본변동표
final_IS = sep('IS', final_reprt_df) #손익계산서

#### 재무상태표에서 자산총계, 자본총계, 부채총계 가져오기

In [181]:
final_CIS.columns

Index(['접수번호', '보고서코드', '사업연도', '고유번호', '재무제표구분', '재무제표명', '계정ID', '계정명',
       '계정상세', '당기명', '당기금액', '당기누적금액', '전기명', '전기금액', '전기명(분/반기)',
       '전기금액(분/반기)', '전기누적금액', '전전기명', '전전기금액', '계정과목정렬순서'],
      dtype='object')

In [182]:
final_BS

,접수번호,보고서코드,사업연도,고유번호,재무제표구분,재무제표명,계정ID,계정명,계정상세,당기명,당기금액,당기누적금액,전기명,전기금액,전기명(분/반기),전기금액(분/반기),전기누적금액,전전기명,전전기금액,계정과목정렬순서
0,20230811002512,11012,2023,00351579,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 27 기 반기말,21056567798,,제 26 기말,25515476742,,,,,,1
1,20230811002512,11012,2023,00351579,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 27 기 반기말,112003998,,제 26 기말,4725764715,,,,,,2
2,20230811002512,11012,2023,00351579,BS,재무상태표,ifrs-full_TradeAndOtherCurrentReceivables,매출채권 및 기타유동채권,-,제 27 기 반기말,14942394354,,제 26 기말,10032132051,,,,,,3
3,20230811002512,11012,2023,00351579,BS,재무상태표,ifrs-full_CurrentFinancialAssetsAtFairValueThr...,당기손익-공정가치금융자산,-,제 27 기 반기말,1056584869,,제 26 기말,1199590459,,,,,,4
4,20230811002512,11012,2023,00351579,BS,재무상태표,ifrs-full_OtherCurrentFinancialAssets,기타금융자산,-,제 27 기 반기말,1340514405,,제 26 기말,6385742957,,,,,,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,20230814001719,11012,2023,00335076,BS,재무상태표,dart_CapitalSurplus,자본잉여금,-,제 32 기 반기말,24224723424,,제 31 기말,24169325128,,,,,,39
37,20230814001719,11012,2023,00335076,BS,재무상태표,dart_OtherCapitalAdjustments,기타자본조정,-,제 32 기 반기말,635677911,,제 31 기말,673516837,,,,,,40
38,20230814001719,11012,2023,00335076,BS,재무상태표,ifrs-full_RetainedEarnings,이익잉여금(결손금),-,제 32 기 반기말,26309405584,,제 31 기말,25144343348,,,,,,41
39,20230814001719,11012,2023,00335076,BS,재무상태표,ifrs-full_Equity,자본총계,-,제 32 기 반기말,66880998419,,제 31 기말,65698376813,,,,,,42


In [183]:
# account_list = final_CIS['계정명'].unique()


# keywords = ['매출', '순이익', '분기', '반기', '당기', '수익', '영업']

# # 키워드가 포함된 항목 필터링
# filtered_accounts = [account for account in account_list if any(keyword in account for keyword in keywords)]

# # 결과 출력
# print(filtered_accounts)

In [184]:
bs_info_df = final_BS[(final_BS['계정명'] == '자산총계') | (final_BS['계정명'] == '자산 총계')
| (final_BS['계정명'] == '부채총계') | (final_BS['계정명'] == '부채 총계') | (final_BS['계정명'] == '자본총계') |
                      (final_BS['계정명'] == '자본 총계')][['고유번호', '재무제표명', "계정명", '전전기명', '전전기금액','전기명', '전기금액', '당기명', '당기금액'
                      ]]
display(bs_info_df)
print(bs_info_df.info())

bs_info_df = bs_info_df.merge(sorted_filtered_df[['고유번호', '회사명']], on='고유번호', how='left')
cols = [bs_info_df.columns[-1]] + list(bs_info_df.columns[1:-1]) + [bs_info_df.columns[0]]
bs_info_df = bs_info_df[cols]
bs_info_df['계정명'] = bs_info_df['계정명'].str.replace(r'\s+', '', regex=True)
bs_info_df

,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액
19,00351579,재무상태표,자산총계,,,제 26 기말,73150553984,제 27 기 반기말,67128236950
34,00351579,재무상태표,부채총계,,,제 26 기말,26975420868,제 27 기 반기말,24709369292
40,00351579,재무상태표,자본총계,,,제 26 기말,46175133116,제 27 기 반기말,42418867658
22,00238782,재무상태표,자산총계,,,제 31 기말,461927087452,제 32 기 반기말,433147427592
39,00238782,재무상태표,부채총계,,,제 31 기말,265438689364,제 32 기 반기말,236543272212
44,00238782,재무상태표,자본총계,,,제 31 기말,196488398088,제 32 기 반기말,196604155380
22,00204262,재무상태표,자산총계,,,제 33 기말,390682573764,제 34 기 반기말,414999908069
34,00204262,재무상태표,부채총계,,,제 33 기말,69065435597,제 34 기 반기말,61754986765
40,00204262,재무상태표,자본총계,,,제 33 기말,321617138167,제 34 기 반기말,353244921304
17,00445841,재무상태표,자산총계,,,제 24 기말,131274219293,제 25 기 반기말,136570806436


<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 19 to 39
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   고유번호    29 non-null     object
 1   재무제표명   29 non-null     object
 2   계정명     29 non-null     object
 3   전전기명    29 non-null     object
 4   전전기금액   29 non-null     object
 5   전기명     29 non-null     object
 6   전기금액    29 non-null     object
 7   당기명     29 non-null     object
 8   당기금액    29 non-null     object
dtypes: object(9)
memory usage: 3.3+ KB
None


,회사명,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호
0,SGA,재무상태표,자산총계,,,제 26 기말,73150553984,제 27 기 반기말,67128236950,00351579
1,SGA,재무상태표,부채총계,,,제 26 기말,26975420868,제 27 기 반기말,24709369292,00351579
2,SGA,재무상태표,자본총계,,,제 26 기말,46175133116,제 27 기 반기말,42418867658,00351579
3,다우데이타,재무상태표,자산총계,,,제 31 기말,461927087452,제 32 기 반기말,433147427592,00238782
4,다우데이타,재무상태표,부채총계,,,제 31 기말,265438689364,제 32 기 반기말,236543272212,00238782
5,다우데이타,재무상태표,자본총계,,,제 31 기말,196488398088,제 32 기 반기말,196604155380,00238782
6,한글과컴퓨터,재무상태표,자산총계,,,제 33 기말,390682573764,제 34 기 반기말,414999908069,00204262
7,한글과컴퓨터,재무상태표,부채총계,,,제 33 기말,69065435597,제 34 기 반기말,61754986765,00204262
8,한글과컴퓨터,재무상태표,자본총계,,,제 33 기말,321617138167,제 34 기 반기말,353244921304,00204262
9,MDS테크,재무상태표,자산총계,,,제 24 기말,131274219293,제 25 기 반기말,136570806436,00445841


In [185]:
total_comp_set = set(sorted_filtered_df['회사명'].unique())
bs_comp_set = set(bs_info_df['회사명'].unique())
print(total_comp_set - bs_comp_set)

set()


In [186]:
# final_BS[final_BS['고유번호'] == '00620868'] 2023년 반기보고서 아이티센 오류

In [187]:
기업명_순서 = list(sorted_filtered_df['회사명'])
계정명_순서 = ['자산총계', '부채총계', '자본총계']


# 기업명과 계정명에 따라 모든 조합을 포함하는 데이터프레임 생성
expected_rows = pd.DataFrame([(company, account) for company in 기업명_순서 for account in 계정명_순서],
                             columns=['회사명', '계정명'])

# 기존 데이터와 병합, 없는 데이터는 '업로드 없음'으로 채우기
bs_info_df = expected_rows.merge(bs_info_df, on=['회사명', '계정명'], how='left').fillna({'값': '업로드 없음'})

# '회사명'과 '계정명'을 Categorical로 변환하여 정렬 기준 적용
bs_info_df['회사명'] = pd.Categorical(bs_info_df['회사명'], categories=기업명_순서, ordered=True)
bs_info_df['계정명'] = pd.Categorical(bs_info_df['계정명'], categories=계정명_순서, ordered=True)

# 정렬 적용
bs_info_df = bs_info_df.sort_values(by=['회사명', '계정명']).reset_index(drop=True)

bs_info_df['행 개수'] = bs_info_df.groupby('회사명')['회사명'].transform('count')
display(bs_info_df)

# # '기업명'과 '계정명'을 Categorical로 변환하여 정렬 기준 적용
# bs_info_df['회사명'] = pd.Categorical(bs_info_df['회사명'], categories=기업명_순서, ordered=True)
# bs_info_df['계정명'] = pd.Categorical(bs_info_df['계정명'], categories=계정명_순서, ordered=True)

# # 정렬 적용
# bs_info_df = bs_info_df.sort_values(by=['회사명', '계정명']).reset_index(drop=True)
# bs_info_df['행 개수'] = bs_info_df.groupby('회사명')['회사명'].transform('count')
# display(bs_info_df)

<ipython-input-187-2c8fda638b4e>:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bs_info_df['행 개수'] = bs_info_df.groupby('회사명')['회사명'].transform('count')


,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수
0,SGA,자산총계,재무상태표,,,제 26 기말,73150553984,제 27 기 반기말,67128236950,00351579,3
1,SGA,부채총계,재무상태표,,,제 26 기말,26975420868,제 27 기 반기말,24709369292,00351579,3
2,SGA,자본총계,재무상태표,,,제 26 기말,46175133116,제 27 기 반기말,42418867658,00351579,3
3,다우데이타,자산총계,재무상태표,,,제 31 기말,461927087452,제 32 기 반기말,433147427592,00238782,3
4,다우데이타,부채총계,재무상태표,,,제 31 기말,265438689364,제 32 기 반기말,236543272212,00238782,3
5,다우데이타,자본총계,재무상태표,,,제 31 기말,196488398088,제 32 기 반기말,196604155380,00238782,3
6,한글과컴퓨터,자산총계,재무상태표,,,제 33 기말,390682573764,제 34 기 반기말,414999908069,00204262,3
7,한글과컴퓨터,부채총계,재무상태표,,,제 33 기말,69065435597,제 34 기 반기말,61754986765,00204262,3
8,한글과컴퓨터,자본총계,재무상태표,,,제 33 기말,321617138167,제 34 기 반기말,353244921304,00204262,3
9,MDS테크,자산총계,재무상태표,,,제 24 기말,131274219293,제 25 기 반기말,136570806436,00445841,3


#### 포괄손익계산서에서 매출액, 영업이익, 당기순이익(분기순이익) 가져오기

In [188]:
# print(final_CIS.columns)
CIS_info_df = final_CIS[(final_CIS['계정명'] == "매출액") | (final_CIS['계정명'] == "수익(매출액)") |
                        (final_CIS['계정명'] == "영업수익") |(final_CIS['계정명'] == "영업이익(손실)") |
                         (final_CIS['계정명'] == "영업이익") | (final_CIS['계정명'] == "분기순이익(손실)") |
                          (final_CIS['계정명'] == "분기순이익") | (final_CIS['계정명'] == "당기순이익(손실)" )|
                           (final_CIS['계정명'] == "당기순이익") | (final_CIS['계정명'] == "반기순이익(손실)") | (final_CIS['계정명'] == "반기순이익")][['사업연도', '고유번호', '재무제표명', '계정명', '전전기명','전전기금액','전기명', '전기금액', '전기금액(분/반기)','전기누적금액',
                           '당기명','당기금액', '당기누적금액']]
# CIS_info_df.info()
# pd.set_option('display.max_rows', None)  # 모든 행 출력


CIS_info_df = CIS_info_df.merge(sorted_filtered_df[['고유번호', '회사명']], on='고유번호', how='left')
cols = [CIS_info_df.columns[-1]] + list(CIS_info_df.columns[1:-1]) + [CIS_info_df.columns[0]]
CIS_info_df = CIS_info_df[cols]
# 문자열 앞뒤 공백 제거
CIS_info_df['계정명'] = CIS_info_df['계정명'].str.strip()
CIS_info_df['계정명'] = CIS_info_df['계정명'].str.replace(r'\s+', '', regex=True)

# 조건에 따라 값 변경
CIS_info_df['계정명'] = CIS_info_df['계정명'].apply(lambda x:
    '매출액' if '매출액' in x or '영업수익' in x else
    '영업이익' if '영업이익' in x else
    '당기순이익' if '순이익' in x else x
)

display(CIS_info_df)
print(CIS_info_df.info())

,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기누적금액,당기명,당기누적금액,사업연도
0,SGA,00351579,포괄손익계산서,매출액,,,,20304792999,제 27 기 반기,19558897970,2023
1,SGA,00351579,포괄손익계산서,영업이익,,,,-827748612,제 27 기 반기,-3211734029,2023
2,SGA,00351579,포괄손익계산서,당기순이익,,,,431956389,제 27 기 반기,-3756265458,2023
3,다우데이타,00238782,포괄손익계산서,매출액,,,,102135003951,제 32 기 반기,86811864862,2023
4,다우데이타,00238782,포괄손익계산서,영업이익,,,,4372419819,제 32 기 반기,3268568105,2023
5,다우데이타,00238782,포괄손익계산서,당기순이익,,,,12494069019,제 32 기 반기,11605757292,2023
6,한글과컴퓨터,00204262,포괄손익계산서,매출액,,,,64771505436,제 34 기 반기,63747512465,2023
7,한글과컴퓨터,00204262,포괄손익계산서,영업이익,,,,23933809114,제 34 기 반기,23386587648,2023
8,한글과컴퓨터,00204262,포괄손익계산서,당기순이익,,,,31086582040,제 34 기 반기,25324939457,2023
9,MDS테크,00445841,포괄손익계산서,매출액,,,,45100994416,제 25 기 반기,43782535780,2023


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   회사명     26 non-null     object
 1   고유번호    26 non-null     object
 2   재무제표명   26 non-null     object
 3   계정명     26 non-null     object
 4   전전기명    26 non-null     object
 5   전전기금액   26 non-null     object
 6   전기명     26 non-null     object
 7   전기누적금액  26 non-null     object
 8   당기명     26 non-null     object
 9   당기누적금액  26 non-null     object
 10  사업연도    26 non-null     object
dtypes: object(11)
memory usage: 2.4+ KB
None


### 포괄손익계산서 대신 손익계산서를 작성하여 해당 정보를 못가져오는 경우가 있다

In [189]:
CIS_info_df['행 개수'] = CIS_info_df.groupby('회사명')['회사명'].transform('count')
display(CIS_info_df[CIS_info_df['행 개수'] != 3])
print("매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------")
print(CIS_info_df[CIS_info_df['행 개수'] != 3]['회사명'].unique())

,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기누적금액,당기명,당기누적금액,사업연도,행 개수
15,콤텍시스템,00186559,포괄손익계산서,매출액,,,,62064646531,제 41 기 반기,105208947407,2023,1
16,쌍용정보통신,00138303,포괄손익계산서,당기순이익,,,,-2230855552,제 43 기 반기,3524682066,2023,1


매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------
['콤텍시스템' '쌍용정보통신']


### 손익계산서에서 해당 기업 정보를 가져와서 포괄손익계산서 데이터프레임에 붙여준다
- 그리고 다시 정보 뽑아내기

In [190]:
missing_comp = list(CIS_info_df[CIS_info_df['행 개수'] != 3]['고유번호'].unique())
if len(missing_comp)>=1:
  for i in missing_comp:
    try:
      df2 = final_IS[final_IS['고유번호'] == i]
      final_CIS = final_CIS[final_CIS['고유번호'] != i]
      final_CIS = pd.concat([final_CIS, df2], ignore_index=True)
    except:
      print("오류발생!")
else:
  print("모든 기업의 매출액, 영업이익, 당기순이익 크롤링 완료")


# print(final_CIS.columns)
CIS_info_df = final_CIS[(final_CIS['계정명'] == "매출액") | (final_CIS['계정명'] == "수익(매출액)") |
                        (final_CIS['계정명'] == "영업수익") |(final_CIS['계정명'] == "영업이익(손실)") |
                         (final_CIS['계정명'] == "영업이익") | (final_CIS['계정명'] == "분기순이익(손실)") |
                          (final_CIS['계정명'] == "분기순이익") | (final_CIS['계정명'] == "당기순이익(손실)" )|
                           (final_CIS['계정명'] == "당기순이익") | (final_CIS['계정명'] == "반기순이익(손실)") | (final_CIS['계정명'] == "반기순이익")][['사업연도', '고유번호', '재무제표명', '계정명', '전전기명','전전기금액','전기명', '전기금액', '전기금액(분/반기)','전기누적금액',
                           '당기명','당기금액', '당기누적금액']]
# CIS_info_df.info()
# pd.set_option('display.max_rows', None)  # 모든 행 출력


CIS_info_df = CIS_info_df.merge(sorted_filtered_df[['고유번호', '회사명']], on='고유번호', how='left')
cols = [CIS_info_df.columns[-1]] + list(CIS_info_df.columns[1:-1]) + [CIS_info_df.columns[0]]
CIS_info_df = CIS_info_df[cols]
# 문자열 앞뒤 공백 제거
CIS_info_df['계정명'] = CIS_info_df['계정명'].str.strip()
CIS_info_df['계정명'] = CIS_info_df['계정명'].str.replace(r'\s+', '', regex=True)

# 조건에 따라 값 변경
CIS_info_df['계정명'] = CIS_info_df['계정명'].apply(lambda x:
    '매출액' if '매출액' in x or '영업수익' in x else
    '영업이익' if '영업이익' in x else
    '당기순이익' if '순이익' in x else x
)

display(CIS_info_df)
print(CIS_info_df.info())



,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도
0,SGA,00351579,포괄손익계산서,매출액,,,,,8055826665,20304792999,제 27 기 반기,12536385968,19558897970,2023
1,SGA,00351579,포괄손익계산서,영업이익,,,,,-1698836587,-827748612,제 27 기 반기,-1485089746,-3211734029,2023
2,SGA,00351579,포괄손익계산서,당기순이익,,,,,-755578742,431956389,제 27 기 반기,-1531130946,-3756265458,2023
3,다우데이타,00238782,포괄손익계산서,매출액,,,,,43082260837,102135003951,제 32 기 반기,38992140211,86811864862,2023
4,다우데이타,00238782,포괄손익계산서,영업이익,,,,,1559821577,4372419819,제 32 기 반기,1480621124,3268568105,2023
5,다우데이타,00238782,포괄손익계산서,당기순이익,,,,,448826459,12494069019,제 32 기 반기,-1012732164,11605757292,2023
6,한글과컴퓨터,00204262,포괄손익계산서,매출액,,,,,33613353246,64771505436,제 34 기 반기,35042034210,63747512465,2023
7,한글과컴퓨터,00204262,포괄손익계산서,영업이익,,,,,13909742574,23933809114,제 34 기 반기,14132294337,23386587648,2023
8,한글과컴퓨터,00204262,포괄손익계산서,당기순이익,,,,,18153235833,31086582040,제 34 기 반기,17771033298,25324939457,2023
9,MDS테크,00445841,포괄손익계산서,매출액,,,,,22255646757,45100994416,제 25 기 반기,21053643162,43782535780,2023


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   회사명         27 non-null     object
 1   고유번호        27 non-null     object
 2   재무제표명       27 non-null     object
 3   계정명         27 non-null     object
 4   전전기명        27 non-null     object
 5   전전기금액       27 non-null     object
 6   전기명         27 non-null     object
 7   전기금액        27 non-null     object
 8   전기금액(분/반기)  27 non-null     object
 9   전기누적금액      27 non-null     object
 10  당기명         27 non-null     object
 11  당기금액        27 non-null     object
 12  당기누적금액      27 non-null     object
 13  사업연도        27 non-null     object
dtypes: object(14)
memory usage: 3.1+ KB
None


### 모든 기업이 매출액, 당기순이익, 영업이익 정보가 있는지 재확인

In [191]:
CIS_info_df['행 개수'] = CIS_info_df.groupby('회사명')['회사명'].transform('count')
display(CIS_info_df[CIS_info_df['행 개수'] != 3])
print("매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------")
print(CIS_info_df[CIS_info_df['행 개수'] != 3]['회사명'].unique())

,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수


매출액, 당기순이익, 영업이익 중 하나 이상 빠진 기업들 -------
[]


### 재무제표, 포괄손익계산서 계정명 통일하기
- 금액 컬럼 데이터타입 숫자로 바꿔주기

In [192]:
bs_info_df['계정명'].unique()

['자산총계', '부채총계', '자본총계']
Categories (3, object): ['자산총계' < '부채총계' < '자본총계']

In [193]:
CIS_info_df['계정명'].unique()

array(['매출액', '영업이익', '당기순이익'], dtype=object)

In [194]:
# df1의 '회사명' 순서를 기준으로 df2 정렬
bs_info_df['회사명'] = pd.Categorical(bs_info_df['회사명'], categories=sorted_filtered_df['회사명'], ordered=True)
bs_info_df = bs_info_df.sort_values('회사명').reset_index(drop=True)
display(bs_info_df)


CIS_info_df['회사명'] = pd.Categorical(CIS_info_df['회사명'], categories=sorted_filtered_df['회사명'], ordered=True)
CIS_info_df = CIS_info_df.sort_values('회사명').reset_index(drop=True)
display(CIS_info_df)


,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수
0,SGA,자산총계,재무상태표,,,제 26 기말,73150553984,제 27 기 반기말,67128236950,00351579,3
1,SGA,부채총계,재무상태표,,,제 26 기말,26975420868,제 27 기 반기말,24709369292,00351579,3
2,SGA,자본총계,재무상태표,,,제 26 기말,46175133116,제 27 기 반기말,42418867658,00351579,3
3,다우데이타,자산총계,재무상태표,,,제 31 기말,461927087452,제 32 기 반기말,433147427592,00238782,3
4,다우데이타,부채총계,재무상태표,,,제 31 기말,265438689364,제 32 기 반기말,236543272212,00238782,3
5,다우데이타,자본총계,재무상태표,,,제 31 기말,196488398088,제 32 기 반기말,196604155380,00238782,3
6,한글과컴퓨터,자산총계,재무상태표,,,제 33 기말,390682573764,제 34 기 반기말,414999908069,00204262,3
7,한글과컴퓨터,부채총계,재무상태표,,,제 33 기말,69065435597,제 34 기 반기말,61754986765,00204262,3
8,한글과컴퓨터,자본총계,재무상태표,,,제 33 기말,321617138167,제 34 기 반기말,353244921304,00204262,3
9,MDS테크,자산총계,재무상태표,,,제 24 기말,131274219293,제 25 기 반기말,136570806436,00445841,3


,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수
0,SGA,00351579,포괄손익계산서,매출액,,,,,8055826665,20304792999,제 27 기 반기,12536385968,19558897970,2023,3
1,SGA,00351579,포괄손익계산서,영업이익,,,,,-1698836587,-827748612,제 27 기 반기,-1485089746,-3211734029,2023,3
2,SGA,00351579,포괄손익계산서,당기순이익,,,,,-755578742,431956389,제 27 기 반기,-1531130946,-3756265458,2023,3
3,다우데이타,00238782,포괄손익계산서,매출액,,,,,43082260837,102135003951,제 32 기 반기,38992140211,86811864862,2023,3
4,다우데이타,00238782,포괄손익계산서,영업이익,,,,,1559821577,4372419819,제 32 기 반기,1480621124,3268568105,2023,3
5,다우데이타,00238782,포괄손익계산서,당기순이익,,,,,448826459,12494069019,제 32 기 반기,-1012732164,11605757292,2023,3
6,한글과컴퓨터,00204262,포괄손익계산서,매출액,,,,,33613353246,64771505436,제 34 기 반기,35042034210,63747512465,2023,3
7,한글과컴퓨터,00204262,포괄손익계산서,영업이익,,,,,13909742574,23933809114,제 34 기 반기,14132294337,23386587648,2023,3
8,한글과컴퓨터,00204262,포괄손익계산서,당기순이익,,,,,18153235833,31086582040,제 34 기 반기,17771033298,25324939457,2023,3
9,MDS테크,00445841,포괄손익계산서,매출액,,,,,22255646757,45100994416,제 25 기 반기,21053643162,43782535780,2023,3


In [195]:


# 숫자가 아닌 값(공백 등)을 NaN으로 변환한 후, int로 변환
# cols = ['당기금액', '전기금액', '전기금액(분/반기)', '전전기금액']
# CIS_info_df[cols] = CIS_info_df[cols].apply(lambda x: pd.to_numeric(x.str.strip().replace('', 'NaN'), errors='coerce'))

# cols = ['당기금액', '전기금액', '전전기금액']
# bs_info_df[cols] = bs_info_df[cols].apply(lambda x: pd.to_numeric(x.str.strip().replace('', 'NaN'), errors='coerce'))

# print(CIS_info_df.info())
# print(bs_info_df.info())

In [196]:
display(bs_info_df)
display(CIS_info_df)

,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수
0,SGA,자산총계,재무상태표,,,제 26 기말,73150553984,제 27 기 반기말,67128236950,00351579,3
1,SGA,부채총계,재무상태표,,,제 26 기말,26975420868,제 27 기 반기말,24709369292,00351579,3
2,SGA,자본총계,재무상태표,,,제 26 기말,46175133116,제 27 기 반기말,42418867658,00351579,3
3,다우데이타,자산총계,재무상태표,,,제 31 기말,461927087452,제 32 기 반기말,433147427592,00238782,3
4,다우데이타,부채총계,재무상태표,,,제 31 기말,265438689364,제 32 기 반기말,236543272212,00238782,3
5,다우데이타,자본총계,재무상태표,,,제 31 기말,196488398088,제 32 기 반기말,196604155380,00238782,3
6,한글과컴퓨터,자산총계,재무상태표,,,제 33 기말,390682573764,제 34 기 반기말,414999908069,00204262,3
7,한글과컴퓨터,부채총계,재무상태표,,,제 33 기말,69065435597,제 34 기 반기말,61754986765,00204262,3
8,한글과컴퓨터,자본총계,재무상태표,,,제 33 기말,321617138167,제 34 기 반기말,353244921304,00204262,3
9,MDS테크,자산총계,재무상태표,,,제 24 기말,131274219293,제 25 기 반기말,136570806436,00445841,3


,회사명,고유번호,재무제표명,계정명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수
0,SGA,00351579,포괄손익계산서,매출액,,,,,8055826665,20304792999,제 27 기 반기,12536385968,19558897970,2023,3
1,SGA,00351579,포괄손익계산서,영업이익,,,,,-1698836587,-827748612,제 27 기 반기,-1485089746,-3211734029,2023,3
2,SGA,00351579,포괄손익계산서,당기순이익,,,,,-755578742,431956389,제 27 기 반기,-1531130946,-3756265458,2023,3
3,다우데이타,00238782,포괄손익계산서,매출액,,,,,43082260837,102135003951,제 32 기 반기,38992140211,86811864862,2023,3
4,다우데이타,00238782,포괄손익계산서,영업이익,,,,,1559821577,4372419819,제 32 기 반기,1480621124,3268568105,2023,3
5,다우데이타,00238782,포괄손익계산서,당기순이익,,,,,448826459,12494069019,제 32 기 반기,-1012732164,11605757292,2023,3
6,한글과컴퓨터,00204262,포괄손익계산서,매출액,,,,,33613353246,64771505436,제 34 기 반기,35042034210,63747512465,2023,3
7,한글과컴퓨터,00204262,포괄손익계산서,영업이익,,,,,13909742574,23933809114,제 34 기 반기,14132294337,23386587648,2023,3
8,한글과컴퓨터,00204262,포괄손익계산서,당기순이익,,,,,18153235833,31086582040,제 34 기 반기,17771033298,25324939457,2023,3
9,MDS테크,00445841,포괄손익계산서,매출액,,,,,22255646757,45100994416,제 25 기 반기,21053643162,43782535780,2023,3


In [197]:
# 기업명_순서 = list(sorted_filtered_df['회사명'])
# 계정명_순서 = ['자산총계', '부채총계', '자본총계']

# # '기업명'과 '계정명'을 Categorical로 변환하여 정렬 기준 적용
# bs_info_df['회사명'] = pd.Categorical(bs_info_df['회사명'], categories=기업명_순서, ordered=True)
# bs_info_df['계정명'] = pd.Categorical(bs_info_df['계정명'], categories=계정명_순서, ordered=True)

# # 정렬 적용
# bs_info_df = bs_info_df.sort_values(by=['회사명', '계정명']).reset_index(drop=True)
# display(bs_info_df)

In [198]:
기업명_순서 = list(sorted_filtered_df['회사명'])
계정명_순서 = [ '매출액', '영업이익', '당기순이익']


# 기업명과 계정명에 따라 모든 조합을 포함하는 데이터프레임 생성
expected_rows = pd.DataFrame([(company, account) for company in 기업명_순서 for account in 계정명_순서],
                             columns=['회사명', '계정명'])

# 기존 데이터와 병합, 없는 데이터는 '업로드 없음'으로 채우기
CIS_info_df = expected_rows.merge(CIS_info_df, on=['회사명', '계정명'], how='left').fillna({'값': '업로드 없음'})

# '회사명'과 '계정명'을 Categorical로 변환하여 정렬 기준 적용
CIS_info_df['회사명'] = pd.Categorical(CIS_info_df['회사명'], categories=기업명_순서, ordered=True)
CIS_info_df['계정명'] = pd.Categorical(CIS_info_df['계정명'], categories=계정명_순서, ordered=True)

# 정렬 적용
CIS_info_df = CIS_info_df.sort_values(by=['회사명', '계정명']).reset_index(drop=True)


display(CIS_info_df)

# # '기업명'과 '계정명'을 Categorical로 변환하여 정렬 기준 적용
# CIS_info_df['회사명'] = pd.Categorical(CIS_info_df['회사명'], categories=기업명_순서, ordered=True)
# CIS_info_df['계정명'] = pd.Categorical(CIS_info_df['계정명'], categories=계정명_순서, ordered=True)

# # 정렬 적용
# CIS_info_df = CIS_info_df.sort_values(by=['회사명', '계정명']).reset_index(drop=True)
# display(CIS_info_df)

,회사명,계정명,고유번호,재무제표명,전전기명,전전기금액,전기명,전기금액,전기금액(분/반기),전기누적금액,당기명,당기금액,당기누적금액,사업연도,행 개수
0,SGA,매출액,00351579,포괄손익계산서,,,,,8055826665,20304792999,제 27 기 반기,12536385968,19558897970,2023,3.0
1,SGA,영업이익,00351579,포괄손익계산서,,,,,-1698836587,-827748612,제 27 기 반기,-1485089746,-3211734029,2023,3.0
2,SGA,당기순이익,00351579,포괄손익계산서,,,,,-755578742,431956389,제 27 기 반기,-1531130946,-3756265458,2023,3.0
3,다우데이타,매출액,00238782,포괄손익계산서,,,,,43082260837,102135003951,제 32 기 반기,38992140211,86811864862,2023,3.0
4,다우데이타,영업이익,00238782,포괄손익계산서,,,,,1559821577,4372419819,제 32 기 반기,1480621124,3268568105,2023,3.0
5,다우데이타,당기순이익,00238782,포괄손익계산서,,,,,448826459,12494069019,제 32 기 반기,-1012732164,11605757292,2023,3.0
6,한글과컴퓨터,매출액,00204262,포괄손익계산서,,,,,33613353246,64771505436,제 34 기 반기,35042034210,63747512465,2023,3.0
7,한글과컴퓨터,영업이익,00204262,포괄손익계산서,,,,,13909742574,23933809114,제 34 기 반기,14132294337,23386587648,2023,3.0
8,한글과컴퓨터,당기순이익,00204262,포괄손익계산서,,,,,18153235833,31086582040,제 34 기 반기,17771033298,25324939457,2023,3.0
9,MDS테크,매출액,00445841,포괄손익계산서,,,,,22255646757,45100994416,제 25 기 반기,21053643162,43782535780,2023,3.0


In [199]:
# final_BS[final_BS['고유번호'] == "00620868"]
# BS_info_df


bs_nan_rows = bs_info_df[bs_info_df.isna().any(axis=1)]
cis_nan_rows = CIS_info_df[CIS_info_df.isna().any(axis=1)]
print(bs_nan_rows)
print(cis_nan_rows)

     회사명   계정명 재무제표명 전전기명 전전기금액  전기명 전기금액  당기명 당기금액 고유번호  행 개수
13  아이티센  자본총계   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN     3
      회사명    계정명 고유번호 재무제표명 전전기명 전전기금액  전기명 전기금액 전기금액(분/반기) 전기누적금액  당기명 당기금액  \
15  콤텍시스템    매출액  NaN   NaN  NaN   NaN  NaN  NaN        NaN    NaN  NaN  NaN   
16  콤텍시스템   영업이익  NaN   NaN  NaN   NaN  NaN  NaN        NaN    NaN  NaN  NaN   
17  콤텍시스템  당기순이익  NaN   NaN  NaN   NaN  NaN  NaN        NaN    NaN  NaN  NaN   

   당기누적금액 사업연도  행 개수  
15    NaN  NaN   NaN  
16    NaN  NaN   NaN  
17    NaN  NaN   NaN  


In [209]:

bs_numeric_cols = ['전전기금액', '전기금액', '당기금액']
cis_numeric_cols = ['전전기금액', '전기금액', '전기금액(분/반기)', '전기누적금액', '당기금액', '당기누적금액']

# 숫자형 변환 (에러 발생 시 NaN 처리)
bs_info_df[bs_numeric_cols] = bs_info_df[bs_numeric_cols].apply(pd.to_numeric, errors='coerce')
CIS_info_df[cis_numeric_cols] = CIS_info_df[cis_numeric_cols].apply(pd.to_numeric, errors='coerce')

# 변환 후 데이터 타입 확인
print(bs_info_df.dtypes)
print(CIS_info_df.dtypes)

회사명      category
계정명      category
재무제표명      object
전전기명       object
전전기금액     float64
전기명        object
전기금액      float64
당기명        object
당기금액      float64
고유번호       object
행 개수        int64
dtype: object
회사명           category
계정명           category
고유번호            object
재무제표명           object
전전기명            object
전전기금액          float64
전기명             object
전기금액           float64
전기금액(분/반기)     float64
전기누적금액         float64
당기명             object
당기금액           float64
당기누적금액         float64
사업연도            object
행 개수           float64
dtype: object


In [211]:
bs_info_df

,회사명,계정명,재무제표명,전전기명,전전기금액,전기명,전기금액,당기명,당기금액,고유번호,행 개수
0,SGA,자산총계,재무상태표,,NaN,제 26 기말,7.315055e+10,제 27 기 반기말,6.712824e+10,00351579,3
1,SGA,부채총계,재무상태표,,NaN,제 26 기말,2.697542e+10,제 27 기 반기말,2.470937e+10,00351579,3
2,SGA,자본총계,재무상태표,,NaN,제 26 기말,4.617513e+10,제 27 기 반기말,4.241887e+10,00351579,3
3,다우데이타,자산총계,재무상태표,,NaN,제 31 기말,4.619271e+11,제 32 기 반기말,4.331474e+11,00238782,3
4,다우데이타,부채총계,재무상태표,,NaN,제 31 기말,2.654387e+11,제 32 기 반기말,2.365433e+11,00238782,3
5,다우데이타,자본총계,재무상태표,,NaN,제 31 기말,1.964884e+11,제 32 기 반기말,1.966042e+11,00238782,3
6,한글과컴퓨터,자산총계,재무상태표,,NaN,제 33 기말,3.906826e+11,제 34 기 반기말,4.149999e+11,00204262,3
7,한글과컴퓨터,부채총계,재무상태표,,NaN,제 33 기말,6.906544e+10,제 34 기 반기말,6.175499e+10,00204262,3
8,한글과컴퓨터,자본총계,재무상태표,,NaN,제 33 기말,3.216171e+11,제 34 기 반기말,3.532449e+11,00204262,3
9,MDS테크,자산총계,재무상태표,,NaN,제 24 기말,1.312742e+11,제 25 기 반기말,1.365708e+11,00445841,3


## 엑셀에 넣기
- bs_info_df
- CIS_info_df

In [210]:
import pandas as pd
from openpyxl import load_workbook

# 엑셀 파일 경로
excel_path = "/content/drive/MyDrive/webcrawling/2023년도 2Q 경쟁사 실적현황 (SGA).xlsx"
new_excel_path = "updated_excel.xlsx"  # 새 파일 경로
sheet_name = "23.1Q_2305"

# 기존 엑셀 파일 불러오기 (서식 유지)
wb = load_workbook(excel_path)
ws = wb[sheet_name]

# 데이터프레임 로드 (bs_info_df, CIS_info_df는 미리 정의되어 있어야 함)
# bs_info_df
# CIS_info_df

# 엑셀의 5행과 21행에 있는 회사명 찾기
target_rows = [5, 21]  # openpyxl은 1-based index (pandas는 0-based index)
for row in target_rows:
    for col in range(1, ws.max_column + 1):  # 모든 열 탐색 (openpyxl은 1부터 시작)
        company_name = ws.cell(row=row, column=col).value  # 회사명 가져오기

        if company_name in bs_info_df["회사명"].values:
            try:
                # 특정 계정명의 전기금액 & 당기금액 입력 (openpyxl은 1-based index)
                ws.cell(row=row + 2, column=col, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "자산총계"), "전기금액"].values[0])
                ws.cell(row=row + 3, column=col, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "부채총계"), "전기금액"].values[0])
                ws.cell(row=row + 4, column=col, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "자본총계"), "전기금액"].values[0])
                ws.cell(row=row + 2, column=col + 1, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "자산총계"), "당기금액"].values[0])
                ws.cell(row=row + 3, column=col + 1, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "부채총계"), "당기금액"].values[0])
                ws.cell(row=row + 4, column=col + 1, value=bs_info_df.loc[(bs_info_df["회사명"] == company_name) & (bs_info_df["계정명"] == "자본총계"), "당기금액"].values[0])

                ws.cell(row=row + 6, column=col, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "매출액"), "전기누적금액"].values[0])
                ws.cell(row=row + 7, column=col, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "영업이익"), "전기누적금액"].values[0])
                ws.cell(row=row + 9, column=col, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "당기순이익"), "전기누적금액"].values[0])
                ws.cell(row=row + 6, column=col + 1, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "매출액"), "당기누적금액"].values[0])
                ws.cell(row=row + 7, column=col + 1, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "영업이익"), "당기누적금액"].values[0])
                ws.cell(row=row + 9, column=col + 1, value=CIS_info_df.loc[(CIS_info_df["회사명"] == company_name) & (CIS_info_df["계정명"] == "당기순이익"), "당기누적금액"].values[0])
            except IndexError:
                print(f"데이터 누락: {company_name}의 일부 값이 없습니다.")

# 수정된 데이터를 새 파일로 저장 (서식 유지)
wb.save(new_excel_path)

print(f"업데이트 완료: {new_excel_path} -> '{sheet_name}' 시트에 반영됨 (서식 유지됨)")


업데이트 완료: updated_excel.xlsx -> '23.1Q_2305' 시트에 반영됨 (서식 유지됨)
